In [220]:
import pandas as pd
from collections import defaultdict

# Load the data
df = pd.read_parquet('game_summary.parquet')

In [158]:
unique_teams = list(set(df['team_1_name']).union(df['team_2_name']))
pick_times = {'blue': {'B1': [1], 'B2-B3': [4, 5], 'B4-B5': [8, 9]}, 'red': {'R1-R2': [2, 3], 'R3': [6], 'R4': [7], 'R5': [10]}]}
roles = ['top', 'jng', 'mid', 'bot', 'sup']

In [192]:
flattened_data = []
for team_name in unique_teams:
    for side, prefix in [('blue', 'team_1'), ('red', 'team_2')]:
        team_games = df[df[f"{prefix}_name"] == team_name]
        total_games = len(team_games)
        for role in roles:
            counts = team_games[f"{prefix}_{role}_pick_num"].value_counts()
            pick_time_tuples = list(pick_times[side].items())
            additional_tuples = [('phase_one', range(1, 7)), ('phase_two', range(7, 11))]
            all_tuples = pick_time_tuples + additional_tuples
            
            for pick_group, pick_order in all_tuples:
                total_counts = sum([counts.get(i, 0) for i in pick_order])
                data = {"count": total_counts, "total_games": total_games, "percentage": total_counts / total_games * 100, "team": team_name, "pick_group": pick_group, "role": role, "side": side}
                flattened_data.append(data)
processed_data = pd.DataFrame(flattened_data)

In [197]:
# Importing required libraries
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('League of Legends Draft Analysis'),
    dcc.Dropdown(
        id='team-dropdown',
        options=[
            {'label': team, 'value': team}
            for team in processed_data['team'].unique()
        ],
        value=processed_data['team'].iloc[0],  # Default value
        multi=False  # Single-select
    ),
    dcc.Dropdown(
        id='filter-dropdown',
        options=[
            {'label': 'Side', 'value': 'side'},
            {'label': 'Phase', 'value': 'phase'}
        ],
        value='side',  # Default value
        multi=False  # Single-select
    ),
    dcc.Dropdown(
        id='side-select-dropdown',
        options=[
            {'label': 'Blue', 'value': 'blue'},
            {'label': 'Red', 'value': 'red'}
        ],
        value='blue',  # Default value
        multi=False  # Single-select
    ),
    dcc.Graph(
        id='bar-plot'
    )
])

# Callback to update the Plotly figure based on the dropdown selections
@app.callback(
    Output('bar-plot', 'figure'),
    [Input('team-dropdown', 'value'),
     Input('side-select-dropdown', 'value'),
     Input('filter-dropdown', 'value')]
)
def update_graph(selected_team, side_select, filter_type):
    filtered_df = processed_data[(processed_data['team'] == selected_team) & (processed_data['side'] == side_select)]

    if filter_type == 'side':
        condition = filtered_df['pick_group'].str.contains(r'B1|R1-R2|B2-B3|R3|B4-B5|R4|R5')
    else:  # Phase
        condition = filtered_df['pick_group'].str.contains('phase_one|phase_two')

    filtered_df = filtered_df[condition]

    role_order = ['top', 'jng', 'mid', 'bot', 'sup']

    pick_group_order = [pg for pg in ['B1', 'R1-R2', 'B2-B3', 'R3', 'B4-B5', 'R4', 'R5'] if pg in filtered_df['pick_group'].unique()]

    if filter_type == 'phase' and 'phase_one' in filtered_df['pick_group'].unique():
        pick_group_order = ['phase_one', 'phase_two']

    # Facet using the `facet_col` parameter
    fig = px.bar(filtered_df,
                 x='percentage', y='role', color='pick_group',
                 title=f"{selected_team} {side_select.capitalize()} Side: Role Pick %",
                 labels={'Percentage': '% of Picks'},
                 facet_col='pick_group',  # This will break down the bar charts by pick group
                 category_orders={'role': role_order, 'pick_group': pick_group_order})

    # Adjust the layout to ensure the chart doesn't get too wide
    fig.update_layout(autosize=True, margin=dict(t=50, b=50, l=50, r=50))

    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


In [12]:
import requests as r
champions = r.get('https://ddragon.leagueoflegends.com/cdn/13.20.1/data/en_US/champion.json').json()
champion_map = {key: champions['data'][key]['name'] for key in champions['data'].keys()}
role_map = {'top': 'TOP', 'jng': 'JUNGLE', 'mid': 'MIDDLE', 'bot': 'BOTTOM', 'sup': 'UTILITY'}

In [247]:
unique_teams = list(set(df['team_1_name']).union(df['team_2_name']))
roles = ['top', 'jng', 'mid', 'bot', 'sup']
flattened_data = []
for team_name in unique_teams:
        for role in roles:
            team_1_games = df[df["team_1_name"] == team_name]
            team_2_games = df[df["team_2_name"] == team_name]
            total_counts = team_1_games[f"team_1_{role}"].value_counts().add(team_2_games[f"team_2_{role}"].value_counts(), fill_value=0)
            total_counts = total_counts.sort_values(ascending=False)
            
            for champion, count in total_counts.items():
                data = {"count": count, "champion": champion_map[champion], "team": team_name, "role": role_map[role]}
                flattened_data.append(data)
champion_pick_data = pd.DataFrame(flattened_data)

In [248]:
champion_pick_data.to_csv('champion_pick_data.csv')

In [1]:
import pandas as pd

# Sample dataframes
df1 = pd.DataFrame({'game_time': [3, 1, 4], 'data1': ['a', 'b', 'c']})
df2 = pd.DataFrame({'game_time': [2, 3, 5], 'data2': ['x', 'y', 'z']})

# Concatenate with keys to create a multi-index
combined_df = pd.concat([df1, df2], keys=['df1', 'df2'], sort=False)

# Sort by 'game_time'
sorted_combined_df = combined_df.sort_values(by='game_time')

# Iterate
for (source, _), row in sorted_combined_df.iterrows():
    print(f"From {source}, game_time: {row['game_time']}, data: {row.dropna().to_dict()}")


From df1, game_time: 1, data: {'game_time': 1, 'data1': 'b'}
From df2, game_time: 2, data: {'game_time': 2, 'data2': 'x'}
From df1, game_time: 3, data: {'game_time': 3, 'data1': 'a'}
From df2, game_time: 3, data: {'game_time': 3, 'data2': 'y'}
From df1, game_time: 4, data: {'game_time': 4, 'data1': 'c'}
From df2, game_time: 5, data: {'game_time': 5, 'data2': 'z'}


In [14]:
for champion, data in champions["data"].items():
    with open(f"images/{champion_map[champion]}.png", 'wb') as f:
        f.write(r.get(f"https://ddragon.leagueoflegends.com/cdn/13.19.1/img/champion/{data['image']['full']}").content)